02で前処理をしたデータの読み込みと03で学習したモデルを使って推論を行うためのnotebookです。  

## 必要なライブラリのimport

In [10]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import pickle

In [11]:
# 予測したいデータの読み込み
test = pd.read_csv('../data/processed/processed20240627_test.csv')

## 保存されたモデルを使って予測を行う

In [12]:
n_folds = 5
predictions_lgb = np.zeros(len(test))
predictions_xgb = np.zeros(len(test))
predictions_ctb = np.zeros(len(test))
features = [c for c in test.columns if c not in ['card_id', 'first_active_month']]

for fold_ in range(n_folds):
    # モデルを読み込む
    with open(f'../src/models/model_lgb_fold_{fold_}.pkl', 'rb') as f:
        lgb = pickle.load(f)
    with open(f'../src/models/model_xgb_fold_{fold_}.pkl', 'rb') as f:
        xgb = pickle.load(f)
    with open(f'../src/models/model_ctb_fold_{fold_}.pkl', 'wb') as f:
        ctb = pickle.load(f)
    # モデルで予測を行う
    lgb_predictions = lgb.predict(test[features])
    xgb_predictions = xgb.predict(test[features])
    ctb_predictions = ctb.predict(test[features])

    # 予測結果を足し合わせる
    predictions_lgb += lgb_predictions
    predictions_xgb += xgb_predictions
    predictions_ctb += ctb_predictions

In [ ]:

# 最適な重みを使って最終予測を計算
final_predictions = (predictions_lgb *  +
                     predictions_xgb *  +
                     predictions_ctb * ) / 3

submission = pd.DataFrame({
    'card_id': test['card_id'],
    'target': final_predictions
})
submission.to_csv("../data/row/submit.csv", index=False)


In [13]:
# 予測結果を出力する
print(predictions)

[-4.15466647 -0.37102097 -1.44592203 ...  0.8826852  -4.93927113
  0.154925  ]


## 提出用ファイルの作成

In [15]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("../data/row/submit.csv", index=False)